In [1]:
import pandas as pd
import pypsa
import numpy as np
network = pypsa.Network()

In [30]:
bus = pd.read_csv('data/base_grid/bus.csv')
sub = pd.read_csv('data/base_grid/sub.csv')
sub = sub.drop(columns = 'interconnect')
bus2sub = pd.read_csv('data/base_grid/bus2sub.csv')
bus2sub = bus2sub.drop(columns = 'interconnect')

In [31]:
buses = bus.merge(bus2sub, how = 'left', on='bus_id')
buses = buses.merge(sub,how='left',on='sub_id')

In [32]:
buses['Va'] = buses['Va'].apply(np.deg2rad)
buses['control'] = buses['type'].map({1:'PQ',2:'PV'})
buses.drop(inplace=True,columns = ['type'])
buses.dropna(subset=['control'],inplace=True)

In [33]:
buses = buses.set_index('bus_id')
buses_columns = {'lat':'y',
                                'lon':'x',
                                'Vm':'v_mag_pu',
                                'Pd':'p',
                                'Qd':'q',
                                'Va':'v_ang',
                                'baseKV':'v_nom',
                                'Vmax':'v_mag_pu_max',
                                'Vmin':'v_mag_pu_min'}

buses = buses.rename(columns = buses_columns)
buses = buses[buses_columns.values()]

In [34]:
buses

,y,x,v_mag_pu,p,q,v_ang,v_nom,v_mag_pu_max,v_mag_pu_min
bus_id,,,,,,,,,
1,43.9761,-70.2211,1.03511,0.00,0.00,-1.124168,230.0,1.1,0.9
2,43.9761,-70.2211,1.03706,0.00,0.00,-1.139032,115.0,1.1,0.9
3,43.9761,-70.2211,1.03293,9.97,1.30,-1.165906,18.0,1.1,0.9
4,43.9761,-70.2211,1.03744,0.00,0.00,-1.140197,1.0,1.1,0.9
5,43.9761,-70.2211,1.03607,0.00,0.00,-1.124050,230.0,1.1,0.9
...,...,...,...,...,...,...,...,...,...
3008156,31.0919,-96.6950,1.02611,0.00,0.00,-0.804766,230.0,1.1,0.9
3008157,31.0919,-96.6950,1.02808,0.00,0.00,-0.872862,115.0,1.1,0.9
3008158,30.7217,-96.4608,1.02650,0.00,0.00,-0.688833,500.0,1.1,0.9


In [35]:
network.import_components_from_dataframe(buses,"Bus")

ERROR:pypsa.io:Error, new components for Bus are not unique


In [40]:
generators = pd.read_csv('data/base_grid/plant.csv')
print(generators.columns)
cost = pd.read_csv('data/base_grid/gencost.csv')
generators = generators.merge(cost.drop(columns = ['type','interconnect']),how='left',on='plant_id')
generators = generators.merge(bus.drop(columns = ['type', 'interconnect']),how='left',on='bus_id')


Index(['plant_id', 'bus_id', 'Pg', 'Qg', 'Qmax', 'Qmin', 'Vg', 'mBase',
       'status', 'Pmax', 'Pmin', 'Pc1', 'Pc2', 'Qc1min', 'Qc1max', 'Qc2min',
       'Qc2max', 'ramp_agc', 'ramp_10', 'ramp_30', 'ramp_q', 'apf', 'mu_Pmax',
       'mu_Pmin', 'mu_Qmax', 'mu_Qmin', 'type', 'interconnect', 'GenFuelCost',
       'GenIOB', 'GenIOC', 'GenIOD'],
      dtype='object')


In [41]:
bus.columns

Index(['bus_id', 'type', 'Pd', 'Qd', 'Gs', 'Bs', 'zone_id', 'Vm', 'Va',
       'baseKV', 'loss_zone', 'Vmax', 'Vmin', 'lam_P', 'lam_Q', 'mu_Vmax',
       'mu_Vmin', 'interconnect'],
      dtype='object')

In [42]:
generators.columns

Index(['plant_id', 'bus_id', 'Pg', 'Qg', 'Qmax', 'Qmin', 'Vg', 'mBase',
       'status', 'Pmax', 'Pmin', 'Pc1', 'Pc2', 'Qc1min', 'Qc1max', 'Qc2min',
       'Qc2max', 'ramp_agc', 'ramp_10', 'ramp_30', 'ramp_q', 'apf', 'mu_Pmax',
       'mu_Pmin', 'mu_Qmax', 'mu_Qmin', 'type', 'interconnect', 'GenFuelCost',
       'GenIOB', 'GenIOC', 'GenIOD', 'startup', 'shutdown', 'n', 'c2', 'c1',
       'c0', 'Pd', 'Qd', 'Gs', 'Bs', 'zone_id', 'Vm', 'Va', 'baseKV',
       'loss_zone', 'Vmax', 'Vmin', 'lam_P', 'lam_Q', 'mu_Vmax', 'mu_Vmin'],
      dtype='object')